# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [1]:
# import numpy and pandas

import numpy as np
import pandas as pd
from scipy.stats import ttest_ind
from scipy.stats import ttest_rel
from scipy.stats import ttest_1samp

# Challenge 1 - Independent Sample T-tests

In this challenge, we will be using the Pokemon dataset. Before applying statistical methods to this data, let's first examine the data.

To load the data, run the code below.

In [2]:
# Run this code:

pokemon = pd.read_csv(r'C:\Users\bruna\IronHack\GitHub\data-ft-par-labs\Labs\data\pokemon.csv')

Let's start off by looking at the `head` function in the cell below.

In [3]:
# Your code here:

pokemon.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


In [4]:
pokemon.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   #           800 non-null    int64 
 1   Name        800 non-null    object
 2   Type 1      800 non-null    object
 3   Type 2      414 non-null    object
 4   Total       800 non-null    int64 
 5   HP          800 non-null    int64 
 6   Attack      800 non-null    int64 
 7   Defense     800 non-null    int64 
 8   Sp. Atk     800 non-null    int64 
 9   Sp. Def     800 non-null    int64 
 10  Speed       800 non-null    int64 
 11  Generation  800 non-null    int64 
 12  Legendary   800 non-null    bool  
dtypes: bool(1), int64(9), object(3)
memory usage: 75.9+ KB


The first thing we would like to do is compare the legendary Pokemon to the regular Pokemon. To do this, we should examine the data further. What is the count of legendary vs. non legendary Pokemons?

In [5]:
# Your code here:
pokemon.Legendary.value_counts()


False    735
True      65
Name: Legendary, dtype: int64

Compute the mean and standard deviation of the total points for both legendary and non-legendary Pokemon.

In [4]:
# Your code here:
# 1th Method

pokemon.pivot_table(index = "Legendary", values = 'Total', aggfunc = ["mean", "std", "count"]).round(2)

,mean,std,count
,Total,Total,Total
Legendary,,,
False,417.21,106.76,735
True,637.38,60.94,65


In [6]:
# 2nd Method
pokemon.Legendary = pokemon.Legendary.astype(str)
regular = pokemon.loc[pokemon["Legendary"]=='False']
legendary = pokemon.loc[pokemon["Legendary"]=='True']


In [7]:
mean_reg = np.mean(regular.Total)
mean_leg = np.mean(legendary.Total)
std_reg = np.std(regular.Total)
std_leg = np.std(legendary.Total)

In [8]:
print(f"The mean is : regular pokemon ({round(mean_reg,2)}) x legendary pokemon ({round(mean_leg,2)})")
print(f"The std is : regular pokemon ({round(std_reg,2)}) x legendary pokemon ({round(std_leg,2)})")

The mean is : regular pokemon (417.21) x legendary pokemon (637.38)
The std is : regular pokemon (106.69) x legendary pokemon (60.47)


In [9]:
# 3th Method

pokemon.groupby(pokemon.Legendary)['Total'].agg(['count', 'mean', 'std']).sort_values(['count'], ascending = True).round(2)

,count,mean,std
Legendary,,,
True,65,637.38,60.94
False,735,417.21,106.76


In [10]:
pokemon.groupby("Legendary")["Total"].describe()

,count,mean,std,min,25%,50%,75%,max
Legendary,,,,,,,,
False,735.0,417.213605,106.760417,180.0,324.0,425.0,498.0,700.0
True,65.0,637.384615,60.937389,580.0,580.0,600.0,680.0,780.0


The computation of the mean might give us a clue regarding how the statistical test may turn out; However, it certainly does not prove whether there is a significant difference between the two groups.

In the cell below, use the `ttest_ind` function in `scipy.stats` to compare the total points for legendary and non-legendary Pokemon. Since we do not have any information about the population, assume the variances are not equal.

$Ho$ : There is NOT a significant difference between Legendary Pokemon's and Regular Pokemon's Total scores. 

$Ha$ : There is a significant difference between Legendary Pokemon's and Regular Pokemon's Total scores. 

In [63]:
# Your code here:
alpha = 0.05

hyp_test = ttest_ind(regular.Total, legendary.Total, equal_var = False)


In [64]:
print("Null hypothesis rejected") if hyp_test[1] < alpha else print("Null hypothesis can't be rejected")

print (f"""The t-statistic is {hyp_test[0]} 
The p-value is {hyp_test[1]}""")

Null hypothesis rejected
The t-statistic is -25.8335743895517 
The p-value is 9.357954335957446e-47


What do you conclude from this test? Write your conclusions below.

`Answer:`
    
    Legendary and Regular pokemon are very different based on their Total Score

        

How about we try to compare the different types of pokemon? In the cell below, list the types of Pokemon from column `Type 1` and the count of each type.

In [27]:
# Your code here:

pokemon["Type 1"].value_counts()


Water       112
Normal       98
Grass        70
Bug          69
Psychic      57
Fire         52
Electric     44
Rock         44
Dragon       32
Ghost        32
Ground       32
Dark         31
Poison       28
Fighting     27
Steel        27
Ice          24
Fairy        17
Flying        4
Name: Type 1, dtype: int64

Since water is the largest group of Pokemon, compare the mean and standard deviation of water Pokemon to all other Pokemon.

In [28]:
# Your code here:
# 1th Method
pokemon.groupby(pokemon['Type 1']=='Water')['Total'].agg(['count', 'mean', 'std']).sort_values(['count'], ascending = True).round(2)

,count,mean,std
Type 1,,,
True,112,430.46,113.19
False,688,435.86,121.09


In [29]:
# 2nd Method

pokemon["type"] = np.where(pokemon['Type 1']=='Water', "Water", 'Other') 
pokemon.pivot_table(index = 'type', values='Total', aggfunc = ['mean', 'std']).round(2)

,mean,std
,Total,Total
type,,
Other,435.86,121.09
Water,430.46,113.19


In [30]:
#3th Method

pokemon.loc[pokemon['Type 1']=='Water'].Total.mean()

430.45535714285717

In [31]:
pokemon.loc[pokemon['Type 1']!='Water'].Total.mean()


435.85901162790697

In [32]:
water = pokemon.loc[pokemon['Type 1']=='Water']
other = pokemon.loc[pokemon['Type 1']!='Water'] 


In [34]:
water_mean = np.mean(water.Total)
other_mean = np.mean(other.Total)
print(f"""Mean:
Water type pokemon ({round(water_mean,2)}) 
Other type pokemon ({round(other_mean,2)})""")

Mean:
Water type pokemon (430.46) 
Other type pokemon (435.86)


In [35]:
water_std = np.std(water.Total)
other_std = np.std(other.Total)
print(f"""Std : 
Water type pokemon ({round(water_std,2)}) 
Other type pokemon ({round(other_std,2)})""")

Std : 
Water type pokemon (112.68) 
Other type pokemon (121.0)


In [36]:
pokemon[pokemon["Type 1"]=="Water"].describe().round(2)


,#,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation
count,112.00,112.00,112.00,112.00,112.00,112.00,112.00,112.00,112.00
mean,303.09,430.46,72.06,74.15,72.95,74.81,70.52,65.96,2.86
std,188.44,113.19,27.49,28.38,27.77,29.03,28.46,23.02,1.56
min,7.00,200.00,20.00,10.00,20.00,10.00,20.00,15.00,1.00
25%,130.00,328.75,52.25,53.00,54.50,55.00,50.00,50.00,1.00
50%,275.00,455.00,70.00,72.00,70.00,70.00,65.00,65.00,3.00
75%,456.25,502.25,90.25,92.00,88.50,90.50,89.25,82.00,4.00
max,693.00,770.00,170.00,155.00,180.00,180.00,160.00,122.00,6.00


In [37]:
pokemon[pokemon["Type 1"]!="Water"].describe().round(2)


,#,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation
count,688.00,688.00,688.00,688.00,688.00,688.00,688.00,688.00,688.00
mean,372.54,435.86,68.80,79.79,73.99,72.50,72.13,68.65,3.40
std,209.93,121.09,25.19,33.03,31.72,33.29,27.74,29.93,1.67
min,1.00,180.00,1.00,5.00,5.00,10.00,20.00,5.00,1.00
25%,197.75,330.00,50.00,55.00,50.00,45.00,50.00,45.00,2.00
50%,380.00,450.00,65.00,76.00,70.00,65.00,70.00,65.00,3.00
75%,553.25,515.00,80.00,100.00,90.00,95.00,90.00,90.00,5.00
max,721.00,780.00,255.00,190.00,230.00,194.00,230.00,180.00,6.00


Perform a hypothesis test comparing the mean of total points for water Pokemon to all non-water Pokemon. Assume the variances are equal. 

$Ho$ : there is a significant difference between Type Water Pokemon's and Non Type Water Pokemon's Total scores. 

$Ha$ : there NOT is a significant difference between type Water Pokemon's and Non-Water Pokemon's Total scores

In [38]:
water = pokemon.loc[pokemon['type'] == "Water"].Total

In [39]:
other = pokemon.loc[pokemon['type'] == "Other"].Total

In [40]:

hyp_test2 = ttest_ind(water, other)

The t-statistic is -0.4418547448849676 and the p-value is 0.6587140317488793 


In [65]:
print("Null hypothesis rejected") if hyp_test2[1] < alpha else print("Null hypothesis can't be rejected")

print (f"""The t-statistic is {hyp_test2[0]} 
The p-value is {hyp_test2[1]}""")

Null hypothesis can't be rejected
The t-statistic is -0.4418547448849676 
The p-value is 0.6587140317488793


Write your conclusion below.

`Answer:`

    The types are actually not very distinct based on a very high p value



# Challenge 2 - Matched Pairs Test

In this challenge we will compare dependent samples of data describing our Pokemon. Our goal is to see whether there is a significant difference between each Pokemon's defense and attack scores. Our hypothesis is that the defense and attack scores are equal. In the cell below, import the `ttest_rel` function from `scipy.stats` and compare the two columns to see if there is a statistically significant difference between them.

#### Our hypothesis is that the defense and attack scores are equal. 

$Ho$ : There NOT is a significant difference between each Pokemon's defense and attack scores.

$Ha$ : There is a significant difference between each Pokemon's defense and attack scores.

In [67]:
# Your code here:

#Ho : there is a significant difference between each Pokemon's defense and attack scores.
#Ha : there NOT is a significant difference between each Pokemon's defense and attack scores.

hyp_test3 = ttest_rel(pokemon.Attack, pokemon.Defense)
print("Null hypothesis rejected") if hyp_test3[1] < alpha else print("Null hypothesis can't be rejected")

print (f"""The t-statistic is {hyp_test3[0]} 
The p-value is {hyp_test3[1]}""")


Null hypothesis rejected
The t-statistic is 4.325566393330478 
The p-value is 1.7140303479358558e-05


Describe the results of the test in the cell below.

`Answer:`

    The Attack and Defense are actually very statistically distinct based on a very low p value

We are also curious about whether there is a significant difference between the mean of special defense and the mean of special attack. Perform the hypothesis test in the cell below. 

$Ho$ : There NOT is a significant difference between each Pokemon's special defense and special attack scores.

$Ha$ : There is a significant difference between each Pokemon's special defense and special attack scores.

In [68]:
# Your code here:

hyp_test4 = ttest_rel(pokemon['Sp. Atk'], pokemon['Sp. Def'])

print("Null hypothesis rejected") if hyp_test4[1] < alpha else print("Null hypothesis can't be rejected")


print (f"""The t-statistic is {hyp_test4[0]} 
The p-value is {hyp_test4[1]}""")

Null hypothesis can't be rejected
The t-statistic is 0.853986188453353 
The p-value is 0.3933685997548122


Describe the results of the test in the cell below.

`Answer:`

    The Sp.Attack and Sp.Defense are actually not very statistically distinct based on a very high p value

As you may recall, a two sample matched pairs test can also be expressed as a one sample test of the difference between the two dependent columns.

Import the `ttest_1samp` function and perform a one sample t-test of the difference between defense and attack. Test the hypothesis that the difference between the means is zero. Confirm that the results of the test are the same.

$Ho$ : Difference between the means is zero.

$Ha$ : Difference between the means is NOT zero

In [70]:
# Your code here:
    
pokemon_diff = pokemon.eval('Defense-Attack')    
hyp_test5 = ttest_1samp(pokemon_diff, 0)

print("Null hypothesis rejected") if hyp_test5[1] < alpha else print("Null hypothesis can't be rejected")


print (f"""The t-statistic is {hyp_test5[0]} 
The p-value is {hyp_test5[1]}""")

Null hypothesis rejected
The t-statistic is -4.325566393330478 
The p-value is 1.7140303479358558e-05


# Bonus Challenge - The Chi-Square Test

The Chi-Square test is used to determine whether there is a statistically significant difference in frequencies. In other words, we are testing whether there is a relationship between categorical variables or rather when the variables are independent. This test is an alternative to Fisher's exact test and is used in scenarios where sample sizes are larger. However, with a large enough sample size, both tests produce similar results. Read more about the Chi Squared test [here](https://en.wikipedia.org/wiki/Chi-squared_test).

In the cell below, create a contingency table using `pd.crosstab` comparing whether a Pokemon is legenadary or not and whether the Type 1 of a Pokemon is water or not.

In [54]:
# Your code here:

legendary_water = pd.crosstab(pokemon["Legendary"],pokemon["Type 1"]=="Water")
legendary_water

Type 1,False,True
Legendary,,
False,627,108
True,61,4


Perform a chi-squared test using the `chi2_contingency` function in `scipy.stats`. You can read the documentation of the function [here](https://docs.scipy.org/doc/scipy-0.15.1/reference/generated/scipy.stats.chi2_contingency.html).

In [55]:
from scipy.stats import chi2_contingency


In [71]:
# Your code here:

hyp_test6 = chi2_contingency(legendary_water)
print("Null hypothesis rejected") if hyp_test6[1] < alpha else print("Null hypothesis can't be rejected")

print (f"""The t-statistic is {hyp_test6[0]} 
The p-value is {hyp_test6[1]}""")

Null hypothesis can't be rejected
The t-statistic is 2.9429200762850503 
The p-value is 0.08625467249550949


Based on a 95% confidence, should we reject the null hypothesis?

`Answer:`

    Nope, based on a very high p value